In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "你的langchain_key"

os.environ["OPENAI_API_KEY"] = "你的openapi key"
os.environ["OPENAI_BASE_URL"] = "你的openai 请求地址"

In [5]:
pip install  langchain pypdf

Note: you may need to restart the kernel to use updated packages.


In [6]:
# document loader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./DeepSeek_R1.pdf")

pages = []
# 异步加载前 11 页
i = 0  # 手动维护计数器
async for page in loader.alazy_load():
    if i >= 11:  # 只加载前 11 页
        break
    pages.append(page)
    i+=1
    
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'producer': 'macOS 版本13.4（版号22F66） Quartz PDFContext, AppendMode 1.1', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250123075355Z00'00'", 'author': '', 'subject': '', 'trapped': '/False', 'title': '', 'moddate': "D:20250130140144Z00'00'", 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'keywords': '', 'source': './DeepSeek_R1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such

In [7]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [8]:
# text spliter
# 模型类型
# 建议使用官方推荐的第二代嵌入模型：text-embedding-ada-002
# cl100k_base 对应的分词器（TOKENIZER）
from langchain_text_splitters import CharacterTextSplitter
text_splitter_0 = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=20, chunk_overlap=0
)
texts_0 = text_splitter_0.split_text(pages[0].page_content)
print(texts_0[0])

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based o

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 每个片段的最大字符数
    chunk_overlap=100,  # 片段之间的重叠字符数
    separators=["\n\n", "\n"]  # 分割符（按段落、句子、标点等）
)
chunks = text_splitter.split_documents(pages)
print(f"文档被分割成 {len(chunks)} 个块")

文档被分割成 37 个块


In [10]:
print(chunks[0])

page_content='DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models' metadata={'producer': 'macOS 版本13.4（版号22F66） Quar

In [11]:
pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_openai import OpenAIEmbeddings

# 初始化 OpenAI 嵌入模型
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 取第一个片段进行embedding，省token
document_embeddings = embeddings.embed_documents(chunks[0].page_content)
print("文档嵌入：", document_embeddings[0])

文档嵌入： [0.00627337908372283, -0.006705800071358681, 0.003642818657681346, -0.008124272339046001, -0.032601915299892426, 0.0032710020896047354, -0.027832182124257088, -0.023311419412493706, -0.003790234914049506, -0.019393423572182655, 0.008596003986895084, 0.015540946274995804, 0.007744265720248222, -0.009421534836292267, -0.023547284305095673, 0.009749126620590687, 0.03763372451066971, 0.0068204570561647415, 0.01837133802473545, -0.007580469828099012, -0.011839160695672035, -0.00643062312155962, -0.0015200250782072544, -0.007095634005963802, 0.015318183228373528, 0.02248588763177395, 0.017244422808289528, -0.024634888395667076, 0.012926764786243439, -0.008805662393569946, 0.025840425863862038, 0.002158828778192401, -0.016183026134967804, -0.01585543341934681, -0.014020920731127262, -0.005909752566367388, -0.0185023732483387, -0.026102500036358833, 0.001998308813199401, -0.007134945131838322, 0.015645775943994522, 0.022878997027873993, 0.00180339184589684, 0.0007489562849514186, -0.0077

In [13]:
print(len(document_embeddings))

935


In [14]:
# 单个查询
query = "这篇文章介绍DeepSeek的什么版本"
# 使用 embed_query 嵌入单个查询
query_embedding = embeddings.embed_query(query)
print("查询嵌入：", query_embedding)

查询嵌入： [-0.0036168822553008795, 0.0056339893490076065, 0.0115114226937294, -0.007091175764799118, -0.03071567602455616, 0.031049542129039764, -0.025165153667330742, -0.0012815587688237429, -0.027877815067768097, -0.017027167603373528, 0.01893298700451851, 0.02946367859840393, -0.018821697682142258, -0.010315069928765297, -0.004799324553459883, 0.005787011701613665, 0.01564997062087059, -0.011114956811070442, 0.01947551965713501, 0.015330015681684017, 0.010461135767400265, 0.005223612766712904, -0.002417051000520587, -0.029129812493920326, -0.001135492348112166, 0.01369546353816986, 0.009897736832499504, -0.011038445867598057, 0.011970489285886288, -0.0007294625393114984, 0.024149643257260323, -0.019016453996300697, -0.0017241050954908133, -0.013208575546741486, 0.00760936364531517, -0.01584472693502903, -0.016456814482808113, -0.005341856740415096, 0.0049314796924591064, -0.020045874640345573, 0.007574586197733879, 0.014968328177928925, -0.009097849950194359, 0.00348820467479527, 0.0034

In [ ]:
# vector store
from langchain_core.vectorstores import InMemoryVectorStore
# 实例化向量存储
vector_store = InMemoryVectorStore(embeddings)
# 将已经转为向量的文档存储到向量存储中
ids =vector_store.add_documents(documents=chunks)
print(ids)
#vector_store.delete(ids=["93ed319a-5110-40ac-8bf4-117a220ed0cb"])


['b5b11014-7702-45f6-aa0d-7272042c5d4d', '93ed319a-5110-40ac-8bf4-117a220ed0cb', '63b92a2f-bf47-4a10-b66b-cd39776995f7', 'b92098b8-085c-4746-9147-051d4d9b7505', '28b2e320-c5df-4178-8999-95c702738b91', '6e7f6c4d-9509-4af8-b788-50bf1c3625b1', '087ed205-4251-4d15-a500-d62f08adfe7b', '4c867d81-35a4-4572-babd-ae0c96cd26e5', '611eed2d-d9c6-4d12-b83b-d6498e371692', '316b0229-841f-4a45-b0fa-abd48b7f4730', 'eca9f952-45f1-4674-a95a-fbf6bdb6e1fa', '3e8f6991-304f-4dec-a802-324531e1c4c2', '82da94bb-b68d-47f9-8da3-27b4615262cb', 'ea56b97f-059a-47f1-bc5e-c26cc46a7d7d', '8d0e3041-01c4-4f76-8f27-6408cbf95e21', 'c195d2f6-7136-43fb-a596-94b6e11e24da', '3f8e5ae9-5629-4e54-bbdc-74fc2af8b381', 'e079f1d3-2fa6-4d3f-a279-fcd67e06f146', '1cdc3b24-7a25-4207-8229-62666022dac8', '3a7c161b-e2aa-4709-affa-2a9399b695d4', 'ff2851e6-e085-4132-a126-9a2a23d996fb', 'a9342a23-e9f7-4a11-a443-e68ec8158649', '9c956338-ffb0-48dc-97fe-b80d2967ebd7', '8db67bd9-af90-4217-9d0a-5409c8235e21', 'dc4f11a1-b16f-48f9-81c9-059dbd54744a',

In [24]:
vector_store.similarity_search('What is the model introduced in this paper?',k=4)

[Document(id='611eed2d-d9c6-4d12-b83b-d6498e371692', metadata={'producer': 'macOS 版本13.4（版号22F66） Quartz PDFContext, AppendMode 1.1', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250123075355Z00'00'", 'author': '', 'subject': '', 'trapped': '/False', 'title': '', 'moddate': "D:20250130140144Z00'00'", 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'keywords': '', 'source': './DeepSeek_R1.pdf', 'total_pages': 22, 'page': 2, 'page_label': '3'}, page_content='RL on it. This demonstrates that the reasoning patterns discovered by larger base models are cru-\ncial for improving reasoning capabilities. We open-source the distilled Qwen and Llama (Dubey\net al., 2024) series. Notably, our distilled 14B model outperforms state-of-the-art open-source\nQwQ-32B-Preview (Qwen, 2024a) by a large margin, and the distilled 32B and 70B models set a\nnew record on the reasoning benchmarks among dense models.\n3'),
 Document(id='6e7f6

In [25]:
# 将向量存储转换为检索器
retriever = vector_store.as_retriever()
retriever.invoke('What is the model introduced in this paper?')

[Document(id='611eed2d-d9c6-4d12-b83b-d6498e371692', metadata={'producer': 'macOS 版本13.4（版号22F66） Quartz PDFContext, AppendMode 1.1', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250123075355Z00'00'", 'author': '', 'subject': '', 'trapped': '/False', 'title': '', 'moddate': "D:20250130140144Z00'00'", 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'keywords': '', 'source': './DeepSeek_R1.pdf', 'total_pages': 22, 'page': 2, 'page_label': '3'}, page_content='RL on it. This demonstrates that the reasoning patterns discovered by larger base models are cru-\ncial for improving reasoning capabilities. We open-source the distilled Qwen and Llama (Dubey\net al., 2024) series. Notably, our distilled 14B model outperforms state-of-the-art open-source\nQwQ-32B-Preview (Qwen, 2024a) by a large margin, and the distilled 32B and 70B models set a\nnew record on the reasoning benchmarks among dense models.\n3'),
 Document(id='6e7f6